# Comparison with Available Tools

Environment installation

```console
foo@bar:~$ conda create -n r_idtdev python=3.8 hdf5 h5py jupyterlab pip wget pandas numpy tqdm scipy networkx matplotlib seaborn scikit-learn requests pymysql poetry pytables pyyaml nbconvert pandoc ipywidgets anndata rpy2 r-base r-essentials r-recommended r-devtools r-irkernel r-dplyr r-biocmanager r-visnetwork r-readr r-shiny r-stringr r-dt r-rstudioapi r-miniui r-rcpp r-png r-plogr r-rcppeigen r-bh r-jpeg r-lambda.r r-futile.options r-matrixstats r-reticulate r-futile.logger r-snow r-rjson r-deldir r-xml r-rcurl r-chron r-proto r-rsqlite r-gprofiler r-gprofiler2 r-gridextra r-interp r-latticeextra r-formula r-checkmate r-callr r-testthat r-knitr r-rmarkdown -c conda-forge -y
foo@bar:~$ conda activate r_idtdev
foo@bar:~$ pip install gget mygene pybiomart gprofiler-official
foo@bar:~$ jupyter nbextension enable --py widgetsnbextension
foo@bar:~$ ipython kernel install --name r_idtdev --user
foo@bar:~$ cd; mkdir tmp
foo@bar:~$ export TMPDIR=~/tmp
foo@bar:~$ R  # go in the R console
```
```R
# In R console:
list.of.packages <- c("neo2R", "utils", "BED")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)
library(BED)
connectToBed("http://icbedb1:5454")
BiocManager::install(c("biomaRt", "mygene", "basilisk"), update = FALSE)
library(biomaRt)
library(mygene)
library(basilisk)
```

Also, see the *'r_idtdev_environment.yml'* file under *'docs/_notebooks'*.

Summary installed tools:
- Python: `idtrack` requirements
- Python: `biomart`
- Python: `mygene` 
- Python: `gprofiler`
- Python: `gget`
- R: `BED`
- R: `biomart`
- R: `mygene`
- R: `gprofiler`

Initialize the environments

In [1]:
import sys
sys.path.append("../..")

In [2]:
import os
import time
import pandas as pd
import anndata as ad
import tqdm
import pickle
import copy
import random
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import idtrack, mygene, gget, mygene, pybiomart

In [3]:
%load_ext rpy2.ipython

Sun Feb 19 15:24:08 2023 INFO cffi mode is CFFI_MODE.ANY
Sun Feb 19 15:24:08 2023 INFO R home found: /home/icb/kemal.inecik/miniconda3/envs/r_idtdev/lib/R
Sun Feb 19 15:24:09 2023 INFO R library path: 
Sun Feb 19 15:24:09 2023 INFO LD_LIBRARY_PATH: 
Sun Feb 19 15:24:09 2023 INFO Default options to initialize R: rpy2, --quiet, --no-save
Sun 19 Feb 2023 03:24:09 PM CET INFO R is already initialized. No need to initialize.


In [4]:
%%R
suppressMessages(library(mygene))
suppressMessages(library(biomaRt))
suppressMessages(library(gprofiler2))
suppressMessages(library(BED))
connectToBed("http://icbedb1:5454", useCache=TRUE)

Sun 19 Feb 2023 03:24:20 PM CET WARNING R[write to console]: http://icbedb1:5454

Sun 19 Feb 2023 03:24:20 PM CET WARNING R[write to console]: BED

Sun 19 Feb 2023 03:24:20 PM CET WARNING R[write to console]: UCB-Human

Sun 19 Feb 2023 03:24:20 PM CET WARNING R[write to console]: 2019.10.11

Sun 19 Feb 2023 03:24:20 PM CET WARNING R[write to console]: Cache ON



In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [7]:
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

In [ ]:
# !ls -lh /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp

In [ ]:
local_dir = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp"
idt = idtrack.API(local_repository=local_dir)
idt.configure_logger()
idt.initialize_graph(organism_name='homo_sapiens', ensembl_release=107, return_test=True)
idt.calculate_graph_caches()

Demo Runs

In [19]:
%%R
oriId <- c(
    "17237", "105886298", "76429", "80985", "230514", "66459",
    "93696", "72514", "20352", "13347", "100462961", "100043346",
    "12400", "106582", "19062", "245607", "79196", "16878", "320727",
    "230649", "66880", "66245", "103742", "320145", "140795"
)
res <- convBeIds(
    ids=oriId,
    from="Gene",
    from.source="EntrezGene",
    from.org="mouse",
    to.source="Ens_gene",
    restricted=TRUE,
    prefFilter=TRUE
)

In [21]:
%%R
res

        from                 to to.preferred to.entity
1      79196 ENSMUSG00000037606         TRUE   3211593
2      66459 ENSMUSG00000043162         TRUE   3216083
3     320145 ENSMUSG00000048562         TRUE   3218320
4  105886298 ENSMUSG00000090110         TRUE   3220852
5      76429 ENSMUSG00000030946         TRUE   3222042
6      17237 ENSMUSG00000022517         TRUE   3223468
7      13347 ENSMUSG00000028974         TRUE   3232153
8      72514 ENSMUSG00000047632         TRUE   3232384
9      12400 ENSMUSG00000031885         TRUE   3234326
10     20352 ENSMUSG00000030539         TRUE   3238604
11    106582 ENSMUSG00000059791         TRUE   3238742
12    245607 ENSMUSG00000072966         TRUE   3240410
13    230514 ENSMUSG00000035212         TRUE   3247955
14    230649 ENSMUSG00000028710         TRUE   3250036
15     16878 ENSMUSG00000034394         TRUE   3253295
16     66245 ENSMUSG00000063802         TRUE   3256026
17    103742 ENSMUSG00000002580         TRUE   3258341
18     809

In [16]:
%%R
from.id <- "ILMN_1220595"
res <- convBeIds(
   ids=from.id, from="Probe", from.source="GPL6885", from.org="mouse",
   to="Peptide", to.source="Uniprot", to.org="human",
   prefFilter=TRUE
)
res

          from     to to.preferred to.entity
1 ILMN_1220595 Q16552         TRUE   2743903


Random list of identifiers